In [53]:
import warnings
warnings.filterwarnings('ignore')
import random 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h3
import json

In [2]:
resolution = 7
hex_col = 'hex'+str(resolution)
sample_size = 200

# Charger type

In [9]:
df = pd.read_csv('./Bristol charging points.csv')
df = df.groupby('connector2Type').count()
df['connector2Type'] = df.index
df = df[['connector2Type', 'chargeDeviceID']]
df.columns = ['connector2Type','prob']
df['prob'] = df['prob'] / df['prob'].sum()


In [28]:
df_ct =  pd.DataFrame( np.random.choice(a =df['connector2Type'],size=sample_size , p=df['prob']))
df_ct.columns = ['connector2Type']
df_ct

,connector2Type
0,3-pin Type G (BS1363)
1,CCS Type 2 Combo (IEC62196)
2,CCS Type 2 Combo (IEC62196)
3,Type 2 Mennekes (IEC62196)
4,JEVS G105 (CHAdeMO) DC
...,...
195,Type 2 Mennekes (IEC62196)
196,Type 2 Mennekes (IEC62196)
197,Type 2 Mennekes (IEC62196)
198,CCS Type 2 Combo (IEC62196)


# Map

In [11]:
df_map = pd.read_csv('./data_simulation_maps.csv')
df_map['probility'] =(df_map['mean_car'] / df_map['mean_car'].sum() )  
df_map.head()
maps_count = len(df_map)

In [37]:
random_numebrs2 =  np.random.choice(a =np.arange(0, maps_count),size= sample_size, p=df_map['probility'])

df_map.index.rename('ind', inplace=True)
df_map['index'] = df_map.index
p = df_map['index'].isin(random_numebrs2)  
a = df_map[p].copy()
df_map_f = a[['Latitude','Longitude']]
df_map_f

,Latitude,Longitude
ind,,
32040,51.461450,-2.592785
14077,51.490213,-2.620726
24037,51.439638,-2.850858
19525,51.488507,-2.662990
10808,51.443499,-2.577215
...,...,...
42224,51.330993,-2.795120
37972,51.477908,-2.701398
20469,51.375201,-2.863412


# Home

In [39]:
df_h = pd.read_csv('./Bristol_Postcodes.csv')
df_home = df_h[['Postcode' , 'In Use?' , 'Latitude', 'Longitude']]
df_home = df_home[ df_home['In Use?'] == 'Yes' ]
df_m = df_map[[hex_col,'mean_car']]
df_home[hex_col] = df_home.apply(lambda x: h3.geo_to_h3(x.Latitude,x.Longitude,resolution),1)
df_home = df_home.merge(df_m, how='right', on=hex_col) 
df_home['probility'] =((df_home['mean_car']) / (df_home['mean_car'].sum() ))  
df_home.head()

,Postcode,In Use?,Latitude,Longitude,hex7,mean_car,probility
0,BS1 4AG,Yes,51.450513,-2.592989,87195876cffffff,841.78821,7.882127e-08
1,BS1 4AJ,Yes,51.450325,-2.592987,87195876cffffff,841.78821,7.882127e-08
2,BS1 4AN,Yes,51.449857,-2.592995,87195876cffffff,841.78821,7.882127e-08
3,BS1 4AP,Yes,51.449775,-2.593138,87195876cffffff,841.78821,7.882127e-08
4,BS1 4AW,Yes,51.452231,-2.596566,87195876cffffff,841.78821,7.882127e-08


In [40]:
home_count = len(df_home)

In [41]:
random_numebrs3 =  np.random.choice(a =np.arange(0, home_count),size= sample_size, p=df_home['probility'])
df_home['index'] = df_home.index
p = df_home['index'].isin(random_numebrs3)  
a = df_home[p].copy()
df_home = a[['Latitude','Longitude']]
df_home

,Latitude,Longitude
137982,51.452873,-2.590458
247596,51.472729,-2.560638
259702,51.454417,-2.596235
301176,51.503485,-2.596097
416817,51.458457,-2.574165
...,...,...
17043706,51.471579,-2.593291
17178713,51.464748,-2.574533
17293072,51.441133,-2.596625
17609429,51.457904,-2.556339


# Combining

In [61]:
df = df.sample(frac=1)
df_home = df_home.sample(frac=1)
df_map = df_map_f.sample(frac=1)

df = pd.DataFrame( {
                'connector2Type': df_ct['connector2Type'].to_numpy()
            , 'home_lat': df_home['Latitude'].to_numpy()
            , 'home_long': df_home['Longitude'].to_numpy()
            , 'car_lat': df_map['Latitude'].to_numpy()
            , 'car_long': df_map['Longitude'].to_numpy()
            })

# Json

In [62]:
result = []
for _, row in df.iterrows():
    result.append({
        
            'home': {
                'Latitude': row['home_lat'],
                'Longitude': row['home_long'],
            },
            'car': {
                'Latitude': row['car_lat'],
                'Longitude': row['car_long'],
                'connector2Type': row['connector2Type']
            }
       
    })

# Convert the list of dictionaries to JSON
json_output = json.dumps(result, indent=4)

# Save to a file (you can choose your desired filename)
with open('user_data_all_rows.json', 'w') as json_file:
    json_file.write(json_output)

print("Data saved to user_data_all_rows.json")


Data saved to user_data_all_rows.json
